# kerakli kutubxonalarni yuklab olamiz

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

# datasetni yuklab olamiz

In [3]:
df=pd.read_csv('corporate_stress_dataset.csv')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15844\4169159405.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('corporate_stress_dataset.csv')


# dataset haqida umumiy malumotlarni tekshirib kuramiz

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                50000 non-null  int64  
 1   Age                               50000 non-null  int64  
 2   Gender                            50000 non-null  object 
 3   Marital_Status                    50000 non-null  object 
 4   Job_Role                          49999 non-null  object 
 5   Experience_Years                  50000 non-null  int64  
 6   Monthly_Salary_INR                50000 non-null  float64
 7   Working_Hours_per_Week            50000 non-null  int64  
 8   Commute_Time_Hours                49999 non-null  float64
 9   Remote_Work                       50000 non-null  bool   
 10  Stress_Level                      49999 non-null  float64
 11  Health_Issues                     37459 non-null  object 
 12  Comp

# orasiga string kirib qolgan bulishi mumkin ustunlarni tekshirib kuramiz >> sababi boshqa bir taken bn tugagan ustun float 'mental health leave taken' esa object

In [10]:
df['Mental_Health_Leave_Taken']

0        FALSE
1         TRUE
2        FALSE
3         TRUE
4         TRUE
         ...  
49995    False
49996    False
49997    False
49998     True
49999    False
Name: Mental_Health_Leave_Taken, Length: 50000, dtype: object

In [12]:
df['Mental_Health_Leave_Taken'].unique()

array(['FALSE', 'TRUE', '1', nan, '0', True, False], dtype=object)

# bu ustunni tashlab yuboramiz 

In [15]:
df.drop('Mental_Health_Leave_Taken', axis=1, inplace=True)

# target value bulgan 'Burnout_Symptoms' ni ham tekshirib kuramiz

In [18]:
df['Burnout_Symptoms'].value_counts()

Burnout_Symptoms
Occasional    16676
No            16662
Yes           16661
3                 1
Name: count, dtype: int64

#  'Burnout_Symptoms' dagi 3ni almashtirib olamiz

In [21]:
mode_value = df[df['Burnout_Symptoms'] != '3']['Burnout_Symptoms'].mode()[0]
df['Burnout_Symptoms'] = df['Burnout_Symptoms'].replace('3', mode_value)

# yana tekshirib kuramiz

In [24]:
df['Burnout_Symptoms'].value_counts()

Burnout_Symptoms
Occasional    16677
No            16662
Yes           16661
Name: count, dtype: int64

# datasetni boshidagi 5ta qatorni kuzdan kechirib olamiz

In [27]:
df.head()

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Burnout_Symptoms,Location
0,1,56,Non-Binary,Widowed,Developer,5,102703.08,44,1.72,True,...,9.0,2,5,10.0,38,False,True,True,No,Tier-3
1,2,46,Female,Single,Analyst,20,165583.91,54,0.62,True,...,9.0,1,10,9.0,72,True,True,True,No,Tier-3
2,3,32,Female,Single,Developer,10,38005.39,81,1.22,False,...,6.0,1,1,1.0,67,False,False,False,Yes,Metro
3,4,60,Female,Divorced,Analyst,26,51127.36,79,2.03,False,...,8.0,10,5,10.0,98,True,True,False,Yes,Metro
4,5,25,Male,Married,Sales,29,174597.45,63,1.45,True,...,3.0,1,9,2.0,85,True,False,False,No,Tier-1


# missing valuelarni kuzdan keciramiz

In [30]:
df.isnull().sum()

ID                                      0
Age                                     0
Gender                                  0
Marital_Status                          0
Job_Role                                1
Experience_Years                        0
Monthly_Salary_INR                      0
Working_Hours_per_Week                  0
Commute_Time_Hours                      1
Remote_Work                             0
Stress_Level                            1
Health_Issues                       12541
Company_Size                            0
Department                              0
Sleep_Hours                             0
Physical_Activity_Hours_per_Week        0
Manager_Support_Level                   0
Work_Pressure_Level                     1
Annual_Leaves_Taken                     1
Work_Life_Balance                       2
Family_Support_Level                    0
Job_Satisfaction                        0
Performance_Rating                      1
Team_Size                         

# Bush kataklarni tuldiramiz

In [33]:
bush_katak=df.isnull().sum()>0

In [35]:
bush_katak

ID                                  False
Age                                 False
Gender                              False
Marital_Status                      False
Job_Role                             True
Experience_Years                    False
Monthly_Salary_INR                  False
Working_Hours_per_Week              False
Commute_Time_Hours                   True
Remote_Work                         False
Stress_Level                         True
Health_Issues                        True
Company_Size                        False
Department                          False
Sleep_Hours                         False
Physical_Activity_Hours_per_Week    False
Manager_Support_Level               False
Work_Pressure_Level                  True
Annual_Leaves_Taken                  True
Work_Life_Balance                    True
Family_Support_Level                False
Job_Satisfaction                    False
Performance_Rating                   True
Team_Size                         

# umumiy formuladan foydalanib ketamiz

In [38]:
for col in df.columns[bush_katak]:
    if df[col].dtype=='object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].mean(), inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15844\3480344265.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15844\3480344265.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

# hamma ustunlar tulgan tulmaganini tekshirib kuramiz

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                50000 non-null  int64  
 1   Age                               50000 non-null  int64  
 2   Gender                            50000 non-null  object 
 3   Marital_Status                    50000 non-null  object 
 4   Job_Role                          50000 non-null  object 
 5   Experience_Years                  50000 non-null  int64  
 6   Monthly_Salary_INR                50000 non-null  float64
 7   Working_Hours_per_Week            50000 non-null  int64  
 8   Commute_Time_Hours                50000 non-null  float64
 9   Remote_Work                       50000 non-null  bool   
 10  Stress_Level                      50000 non-null  float64
 11  Health_Issues                     50000 non-null  object 
 12  Comp

In [43]:
df.head()

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Burnout_Symptoms,Location
0,1,56,Non-Binary,Widowed,Developer,5,102703.08,44,1.72,True,...,9.0,2,5,10.0,38,False,True,True,No,Tier-3
1,2,46,Female,Single,Analyst,20,165583.91,54,0.62,True,...,9.0,1,10,9.0,72,True,True,True,No,Tier-3
2,3,32,Female,Single,Developer,10,38005.39,81,1.22,False,...,6.0,1,1,1.0,67,False,False,False,Yes,Metro
3,4,60,Female,Divorced,Analyst,26,51127.36,79,2.03,False,...,8.0,10,5,10.0,98,True,True,False,Yes,Metro
4,5,25,Male,Married,Sales,29,174597.45,63,1.45,True,...,3.0,1,9,2.0,85,True,False,False,No,Tier-1


In [45]:
df.isnull().sum()

ID                                  0
Age                                 0
Gender                              0
Marital_Status                      0
Job_Role                            0
Experience_Years                    0
Monthly_Salary_INR                  0
Working_Hours_per_Week              0
Commute_Time_Hours                  0
Remote_Work                         0
Stress_Level                        0
Health_Issues                       0
Company_Size                        0
Department                          0
Sleep_Hours                         0
Physical_Activity_Hours_per_Week    0
Manager_Support_Level               0
Work_Pressure_Level                 0
Annual_Leaves_Taken                 0
Work_Life_Balance                   0
Family_Support_Level                0
Job_Satisfaction                    0
Performance_Rating                  0
Team_Size                           0
Training_Opportunities              0
Gender_Bias_Experienced             0
Discriminati

# Yozuvli ustunlarni topamiz

In [48]:
yozuvli_ustun=df.select_dtypes(include=['object','category']).columns

In [50]:
yozuvli_ustun

Index(['Gender', 'Marital_Status', 'Job_Role', 'Health_Issues', 'Company_Size',
       'Department', 'Burnout_Symptoms', 'Location'],
      dtype='object')

# har bir yozuvli ustunda nectadan unique valuelar borligini tekshirib olamiz >> bu get dummies yoki label encodingni qullashni decide qilish uchun kerak

In [53]:
for col in yozuvli_ustun:
    print(f"column {col}: {df[col].nunique()}")

column Gender: 3
column Marital_Status: 4
column Job_Role: 6
column Health_Issues: 3
column Company_Size: 3
column Department: 6
column Burnout_Symptoms: 3
column Location: 4


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                50000 non-null  int64  
 1   Age                               50000 non-null  int64  
 2   Gender                            50000 non-null  object 
 3   Marital_Status                    50000 non-null  object 
 4   Job_Role                          50000 non-null  object 
 5   Experience_Years                  50000 non-null  int64  
 6   Monthly_Salary_INR                50000 non-null  float64
 7   Working_Hours_per_Week            50000 non-null  int64  
 8   Commute_Time_Hours                50000 non-null  float64
 9   Remote_Work                       50000 non-null  bool   
 10  Stress_Level                      50000 non-null  float64
 11  Health_Issues                     50000 non-null  object 
 12  Comp

# barcha ustunlarni label encodingdan utqazib olamiz

In [58]:
encoder=LabelEncoder()

In [60]:
df[yozuvli_ustun]=df[yozuvli_ustun].apply(encoder.fit_transform)

In [62]:
df.head()

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Burnout_Symptoms,Location
0,1,56,2,3,1,5,102703.08,44,1.72,True,...,9.0,2,5,10.0,38,False,True,True,0,3
1,2,46,0,2,0,20,165583.91,54,0.62,True,...,9.0,1,10,9.0,72,True,True,True,0,3
2,3,32,0,2,1,10,38005.39,81,1.22,False,...,6.0,1,1,1.0,67,False,False,False,2,0
3,4,60,0,0,0,26,51127.36,79,2.03,False,...,8.0,10,5,10.0,98,True,True,False,2,0
4,5,25,1,1,4,29,174597.45,63,1.45,True,...,3.0,1,9,2.0,85,True,False,False,0,1


# booliandan integerga utib olamiz

In [65]:
df[df.columns]=df[df.columns].astype(int)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   ID                                50000 non-null  int32
 1   Age                               50000 non-null  int32
 2   Gender                            50000 non-null  int32
 3   Marital_Status                    50000 non-null  int32
 4   Job_Role                          50000 non-null  int32
 5   Experience_Years                  50000 non-null  int32
 6   Monthly_Salary_INR                50000 non-null  int32
 7   Working_Hours_per_Week            50000 non-null  int32
 8   Commute_Time_Hours                50000 non-null  int32
 9   Remote_Work                       50000 non-null  int32
 10  Stress_Level                      50000 non-null  int32
 11  Health_Issues                     50000 non-null  int32
 12  Company_Size                    

In [69]:
df.head()

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Burnout_Symptoms,Location
0,1,56,2,3,1,5,102703,44,1,1,...,9,2,5,10,38,0,1,1,0,3
1,2,46,0,2,0,20,165583,54,0,1,...,9,1,10,9,72,1,1,1,0,3
2,3,32,0,2,1,10,38005,81,1,0,...,6,1,1,1,67,0,0,0,2,0
3,4,60,0,0,0,26,51127,79,2,0,...,8,10,5,10,98,1,1,0,2,0
4,5,25,1,1,4,29,174597,63,1,1,...,3,1,9,2,85,1,0,0,0,1


# Scaling qilamiz

In [72]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
df = pd.DataFrame(df_scaled, columns=df.columns)


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                50000 non-null  float64
 1   Age                               50000 non-null  float64
 2   Gender                            50000 non-null  float64
 3   Marital_Status                    50000 non-null  float64
 4   Job_Role                          50000 non-null  float64
 5   Experience_Years                  50000 non-null  float64
 6   Monthly_Salary_INR                50000 non-null  float64
 7   Working_Hours_per_Week            50000 non-null  float64
 8   Commute_Time_Hours                50000 non-null  float64
 9   Remote_Work                       50000 non-null  float64
 10  Stress_Level                      50000 non-null  float64
 11  Health_Issues                     50000 non-null  float64
 12  Comp

In [76]:
df.head()

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Burnout_Symptoms,Location
0,-1.732016,1.046863,1.231989,1.345111,-0.877333,-1.273415,-0.142626,-1.146484,-0.006440,0.990683,...,1.264584,-0.947257,0.005025,1.571906,-0.432551,-0.996367,0.993819,1.005576,-1.224910,1.344544
1,-1.731947,0.323961,-1.219438,0.449708,-1.464374,-0.006048,1.064998,-0.528041,-1.226232,0.990683,...,1.264584,-1.263648,1.587305,1.257092,0.744242,1.003647,0.993819,1.005576,-1.224910,1.344544
2,-1.731878,-0.688101,-1.219438,0.449708,-0.877333,-0.850959,-1.385166,1.141754,-0.006440,-1.009404,...,0.316895,-1.263648,-1.260798,-1.261423,0.571184,-0.996367,-1.006219,-0.994455,1.224959,-1.340677
3,-1.731808,1.336023,-1.219438,-1.341100,-1.464374,0.500899,-1.133155,1.018065,1.213351,-1.009404,...,0.948687,1.583874,0.005025,1.571906,1.644142,1.003647,0.993819,-0.994455,1.224959,-1.340677
4,-1.731739,-1.194132,0.006276,-0.445696,0.883790,0.754372,1.238114,0.028557,-0.006440,0.990683,...,-0.630795,-1.263648,1.270849,-0.946609,1.194192,1.003647,-1.006219,-0.994455,-1.224910,-0.445603


# Training qilamiz

In [79]:
X = df.drop(columns=['Burnout_Symptoms'])  # Features
y = df['Burnout_Symptoms']  # Target variable

In [81]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_test,X_val,y_test,y_val=train_test_split(X_temp,y_temp,test_size=0.5,random_state=42)

# Modelni tanlaymiz

In [84]:
model=LinearRegression()

# Modelni Training qildiramiz

In [87]:
model.fit(X_train,y_train)

LinearRegression()

# Model predict 

In [90]:
y_pred = model.predict(X_test)

In [92]:
y

0       -1.224910
1       -1.224910
2        1.224959
3        1.224959
4       -1.224910
           ...   
49995   -1.224910
49996    0.000024
49997   -1.224910
49998    1.224959
49999   -1.224910
Name: Burnout_Symptoms, Length: 50000, dtype: float64

In [94]:
X_train.shape

(40000, 28)

In [96]:
y_train.shape

(40000,)

In [98]:
X_test.shape

(5000, 28)

In [100]:
y_test

47374   -1.224910
48216   -1.224910
45929   -1.224910
48715   -1.224910
18849   -1.224910
           ...   
49939    1.224959
6420    -1.224910
48421    1.224959
26037    0.000024
31247    0.000024
Name: Burnout_Symptoms, Length: 5000, dtype: float64

In [102]:
y_test.shape

(5000,)

In [104]:
y_pred[0]

0.014565843640999755

In [106]:
X_train

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Annual_Leaves_Taken,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Location
39087,0.976011,-1.266422,-1.219438,-0.445696,-0.877333,1.430301,0.598561,0.399623,-0.006440,0.990683,...,-0.558199,-0.946691,-0.314474,1.270849,1.571906,0.190457,1.003647,-1.006219,1.005576,1.344544
30893,0.408314,0.034800,0.006276,0.449708,0.883790,-0.344012,-0.922704,-0.837262,-1.226232,0.990683,...,1.007232,-0.314898,-0.314474,-0.627886,1.257092,-1.332451,-0.996367,-1.006219,-0.994455,1.344544
45278,1.404936,0.323961,1.231989,1.345111,-0.877333,-1.188924,-1.298819,-0.095131,-0.006440,0.990683,...,-0.781833,-0.630795,0.951091,-0.627886,-0.002166,-1.228617,-0.996367,0.993819,-0.994455,-0.445603
16398,-0.595929,-1.700163,-1.219438,-1.341100,0.296749,-0.935450,1.047138,-1.022795,-0.006440,-1.009404,...,1.678132,0.632791,0.634700,1.587305,-0.316980,0.882688,1.003647,-1.006219,-0.994455,1.344544
13653,-0.786109,-0.037490,0.006276,0.449708,0.296749,-1.526888,1.295807,0.337779,-1.226232,0.990683,...,-0.670016,1.264584,-1.263648,0.005025,-0.631795,0.675019,-0.996367,-1.006219,-0.994455,-1.340677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-0.950238,-0.543521,0.006276,-0.445696,0.883790,1.430301,0.342824,-1.517549,-1.226232,0.990683,...,-0.110933,0.316895,1.583874,-1.577254,-0.631795,1.713365,1.003647,0.993819,1.005576,-0.445603
44732,1.367108,-0.254360,1.231989,-0.445696,-0.290292,-0.766468,-0.921264,0.028557,-1.226232,0.990683,...,-1.676365,-0.946691,-1.580039,0.321481,-1.576238,-0.259493,-0.996367,-1.006219,1.005576,1.344544
38158,0.911648,-0.471231,1.231989,1.345111,-0.290292,-0.090539,-0.098493,-0.342508,1.213351,0.990683,...,1.342682,0.948687,1.267483,1.587305,0.942277,0.086622,-0.996367,-1.006219,1.005576,0.449470
860,-1.672434,-1.411003,1.231989,-0.445696,0.883790,-1.104433,-0.964207,1.389131,-0.006440,0.990683,...,1.678132,-0.314898,1.583874,-0.627886,-0.002166,0.709630,1.003647,0.993819,1.005576,0.449470


In [108]:
X_test

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Annual_Leaves_Taken,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Location
47374,1.550151,-1.266422,0.006276,-1.341100,1.470832,-1.357906,0.462338,-0.095131,-0.006440,-1.009404,...,-0.558199,1.580480,0.318309,-0.311431,-0.946609,0.778853,-0.996367,0.993819,-0.994455,-1.340677
48216,1.608486,-1.483293,1.231989,0.449708,-0.290292,0.585390,-0.766873,-0.837262,-0.006440,0.990683,...,0.895416,0.948687,-0.314474,0.954393,-0.316980,-1.574732,1.003647,-1.006219,1.005576,1.344544
45929,1.450038,0.396251,-1.219438,-0.445696,1.470832,-0.597486,-0.287799,1.450975,-1.226232,0.990683,...,-1.676365,0.000998,0.634700,-0.311431,-1.261423,0.882688,-0.996367,0.993819,-0.994455,-1.340677
48715,1.643058,-1.338713,1.231989,0.449708,1.470832,-0.597486,1.264637,0.275934,-1.226232,0.990683,...,0.783599,-1.262587,1.583874,1.270849,0.942277,-0.709443,1.003647,-1.006219,-0.994455,-1.340677
18849,-0.426119,0.613122,0.006276,0.449708,-1.464374,-1.188924,-0.769043,-0.651730,-1.226232,0.990683,...,-0.558199,-0.314898,0.001917,-0.627886,0.312648,1.713365,1.003647,-1.006219,-0.994455,-0.445603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49939,1.727859,-1.049552,0.006276,-1.341100,1.470832,-1.442397,-0.609889,-1.703082,-1.226232,0.990683,...,-1.340915,-0.630795,0.951091,-1.577254,-0.946609,-0.051824,1.003647,-1.006219,1.005576,1.344544
6420,-1.287226,0.902282,0.006276,1.345111,-1.464374,-0.935450,0.162026,-1.208328,-0.006440,-1.009404,...,0.783599,-0.630795,-0.630865,-0.311431,0.312648,0.121234,1.003647,-1.006219,1.005576,-1.340677
48421,1.622689,1.480604,-1.219438,-1.341100,-0.290292,0.838863,-1.516818,1.636508,-0.006440,0.990683,...,-1.564549,1.264584,-1.580039,-0.311431,-0.002166,1.332638,-0.996367,0.993819,-0.994455,1.344544
26037,0.071880,-0.615811,1.231989,-0.445696,-0.877333,-1.357906,0.732844,-0.837262,-0.006440,0.990683,...,-0.222750,1.580480,0.951091,-0.311431,-1.576238,-0.051824,-0.996367,0.993819,1.005576,-1.340677


In [110]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                50000 non-null  float64
 1   Age                               50000 non-null  float64
 2   Gender                            50000 non-null  float64
 3   Marital_Status                    50000 non-null  float64
 4   Job_Role                          50000 non-null  float64
 5   Experience_Years                  50000 non-null  float64
 6   Monthly_Salary_INR                50000 non-null  float64
 7   Working_Hours_per_Week            50000 non-null  float64
 8   Commute_Time_Hours                50000 non-null  float64
 9   Remote_Work                       50000 non-null  float64
 10  Stress_Level                      50000 non-null  float64
 11  Health_Issues                     50000 non-null  float64
 12  Comp

In [112]:
X.head()

,ID,Age,Gender,Marital_Status,Job_Role,Experience_Years,Monthly_Salary_INR,Working_Hours_per_Week,Commute_Time_Hours,Remote_Work,...,Annual_Leaves_Taken,Work_Life_Balance,Family_Support_Level,Job_Satisfaction,Performance_Rating,Team_Size,Training_Opportunities,Gender_Bias_Experienced,Discrimination_Experienced,Location
0,-1.732016,1.046863,1.231989,1.345111,-0.877333,-1.273415,-0.142626,-1.146484,-0.006440,0.990683,...,0.000883,1.264584,-0.947257,0.005025,1.571906,-0.432551,-0.996367,0.993819,1.005576,1.344544
1,-1.731947,0.323961,-1.219438,0.449708,-1.464374,-0.006048,1.064998,-0.528041,-1.226232,0.990683,...,-0.334566,1.264584,-1.263648,1.587305,1.257092,0.744242,1.003647,0.993819,1.005576,1.344544
2,-1.731878,-0.688101,-1.219438,0.449708,-0.877333,-0.850959,-1.385166,1.141754,-0.006440,-1.009404,...,-0.222750,0.316895,-1.263648,-1.260798,-1.261423,0.571184,-0.996367,-1.006219,-0.994455,-1.340677
3,-1.731808,1.336023,-1.219438,-1.341100,-1.464374,0.500899,-1.133155,1.018065,1.213351,-1.009404,...,0.000883,0.948687,1.583874,0.005025,1.571906,1.644142,1.003647,0.993819,-0.994455,-1.340677
4,-1.731739,-1.194132,0.006276,-0.445696,0.883790,0.754372,1.238114,0.028557,-0.006440,0.990683,...,-0.446383,-0.630795,-1.263648,1.270849,-0.946609,1.194192,1.003647,-1.006219,-0.994455,-0.445603


# Modelni baholaymiz

In [119]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [121]:
print(f"\nModel Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.2f}")


Model Evaluation Metrics:
Mean Squared Error (MSE): 1.01
Mean Absolute Error (MAE): 0.83
R² Score: -0.00
